
- 파이썬 언어에서 다음에 대해 조사하고 예제코드를 실행하여 보라

- 파이썬으로 싱글톤 패턴을 구현하는 방법들

- 파이썬에서 멀티쓰레드 구현하는 기법

- 멀티쓰레드에서 공유 변수에 대한 Critical Section을 보호하는 기법

1. new() 메서드를 오버라이드하거나, call() 메서드를 사용하거나, 데코레이터를 활용하는 방식

In [ ]:
#new()

class SingletonNew:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:              # 최초 1번만 생성
            cls._instance = super().__new__(cls)
        return cls._instance

# 테스트
a = SingletonNew()
b = SingletonNew()
print(a is b)  # True → 두 객체는 같은 인스턴스


#__call__()

class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):   # 객체 생성 시 호출
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class SingletonCall(metaclass=SingletonMeta):
    pass

# 테스트
x = SingletonCall()
y = SingletonCall()
print(x is y)  # True

# 데코레이터 활용

def singleton(cls):
    instances = {}

    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]

    return get_instance

@singleton
class SingletonDeco:
    pass

# 테스트
m = SingletonDeco()
n = SingletonDeco()
print(m is n)  # True


2. threading 모듈 사용

In [ ]:
import threading
import time

def worker(name):
    for i in range(3):
        print(f"{name} 작업 중... {i}")
        time.sleep(1)

# 쓰레드 2개 생성
t1 = threading.Thread(target=worker, args=("스레드-1",))
t2 = threading.Thread(target=worker, args=("스레드-2",))

# 시작
t1.start()
t2.start()

# 메인 쓰레드가 기다리도록 join()
t1.join()
t2.join()

print("모든 작업 종료")

3. 뮤텍스(Lock,Rlock)를 사용

In [ ]:
import threading

# 1) 뮤텍스 없이
counter = 0

def worker_no_lock():
    global counter
    for _ in range(100000):
        counter += 1  # 임계구역 (보호 없음)

threads = [threading.Thread(target=worker_no_lock) for _ in range(2)]
for t in threads: t.start()
for t in threads: t.join()

print("최종 counter (뮤텍스 없음):", counter)  # 기대값 200000, 하지만 더 작을 수 있음

# 2) 뮤텍스 사용
counter = 0
lock = threading.Lock()

def worker_with_lock():
    global counter
    for _ in range(100000):
        with lock:       # 임계구역 보호
            counter += 1

threads = [threading.Thread(target=worker_with_lock) for _ in range(2)]
for t in threads: t.start()
for t in threads: t.join()

print("최종 counter (뮤텍스 사용):", counter)  # 항상 200000
